# Train the model for further analysis using Grad_Cam
Here, models are output without a nested layer structure.

The difference between this file and file "main_TL_vs_Scracth":

(1) In file "utils.train_val_test_dataset_import_GC", dataset are rescaled and then put into the model without rescaling layer.

(2) In file "models.TL_models_GC", model are built without rescaling layer and the pre-trained model (e.g., mobilenetv2_1.00_224) are not nested as a layer named "mobilenetv2_1.00_224"

In [ ]:
from utils.miscellaneous import read_config
from copy import deepcopy

import utils.train_val_test_dataset_import_GC as tvt_GC
import utils.class_imbalances as ci
import utils.plots as plot
import models.TL_models_GC as TL
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
import os

import utils.write_excel as write_excel


## Parse configuration file + initializations

In [ ]:
# read config files
cfg = read_config('./config.yaml')

# constants
image_height = cfg['image_height']
image_width = cfg['image_width']
batch_size = cfg['batch_size']['tra']
num_epochs = cfg['trainParams']['num_epochs']
lr_rate = cfg['adamParams']['lr']
num_classes = cfg['num_classes']

# paths
path_train = cfg['Path']['path_train']
path_val = cfg['Path']['path_val']

# load datasets
ds_train, ds_val = tvt_GC.import_dataset_train_val(
    path_train, path_val, image_height, image_width, batch_size)

# autotune
AUTOTUNE = tf.data.AUTOTUNE
ds_train = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
ds_val = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

# class weights
class_weights_train = ci.class_weights_4(path_train)

# paths to model and checkpoint file save
save_model_path_fromscratch = cfg['Path']['save_model_path_fromscratch']
save_ckp_path_fromscratch = cfg['Path']['save_ckp_path_fromscratch']
save_model_path_TL_classifier = cfg['Path']['save_model_path_TL_classifier']
save_ckp_path_TL_classifier = cfg['Path']['save_ckp_path_TL_classifier']
save_model_path_TL_all = cfg['Path']['save_model_path_TL_all']
save_ckp_path_TL_all = cfg['Path']['save_ckp_path_TL_all']

In [ ]:
path_train

In [ ]:
path_val

## Train models

### (1) Experiment 1

In [ ]:

path_val = r"F:\Tianlong\Data\Deep_Plastic\Data\Exp1\Validation 1"

# lr_rate_list = [0.0001]
lr_rate_list = [0.1, 0.01, 0.001, 0.0001, 0.00001]
model_arc_list = ["DenseNet", "ResNet"]
# model_arc_list = ["MobileNet", "SqueezeNet", "ResNet", "Inception", "DenseNet"]
fine_tune_strategy_list = ["TL_all"]
# fine_tune_strategy_list = ["scratch", "TL_classifier","TL_all"]
best_val_acc_list = []
training_time_list = []
model_name_list = []

# path to save weigts
path = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp1\Model_no_rescale_layer\10_times"
# path to save fig
fig_path = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp1\Model_no_rescale_layer\10_times\fig"
# path to save results in a excel file
excel_path=r'F:\Tianlong\Data\Deep_Plastic\Results\Exp1\Model_no_rescale_layer\10_times\models_exp1.xlsx'

In [ ]:
# train models
repeat_times = 1

row_i=1

for fine_tune_strategy in fine_tune_strategy_list:
  for model_arc_name in model_arc_list:
    for lr_rate in lr_rate_list:
      for i in range(repeat_times):
        model_name = model_arc_name + "_" + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + str(i+1) + ".hdf5"
        fig_name_acc = model_arc_name + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + str(i+1) + "_accuracy.jpg"
        fig_name_loss = model_arc_name + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + str(i+1) + "_loss.jpg"
        model_name_list.append(model_name)
        save_model_path = os.path.join(path,model_name)
        save_fig_acc_path = os.path.join(fig_path, fig_name_acc)
        save_fig_loss_path = os.path.join(fig_path, fig_name_loss)

        starttime = datetime.datetime.now()

        if model_arc_name == "SqueezeNet":
          model_result = TL.SqueezeN(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
             save_ckp_path=None, image_height=image_height, image_width=image_width,
             ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)

        if model_arc_name == "ResNet":
          model_result = TL.ResN50(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
            save_ckp_path=None, image_height=image_height, image_width=image_width,
            ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)

        if model_arc_name == "MobileNet":
          model_result = TL.MNetV2(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
          save_ckp_path=None, image_height=image_height, image_width=image_width,
          ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)

        if model_arc_name == "Inception":
          model_result = TL.IncV3(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
            save_ckp_path=None, image_height=image_height, image_width=image_width,
            ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)

        if model_arc_name == "DenseNet":
          model_result = TL.DenseN121(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
            save_ckp_path=None, image_height=image_height, image_width=image_width,
            ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)

        endtime = datetime.datetime.now()

        training_time=round((endtime - starttime).seconds/60)
        print("training_time: ", training_time)
        # training_time_list.append(training_time)

        val_acc = model_result.history['val_accuracy']
        print("Best Validation Accuracy is", max(val_acc))
        # best_val_acc_list.append(max(val_acc))
        
        plot.save_plot_hist(model_result, save_fig_acc_path, save_fig_loss_path)

        write_excel.save_train_results_oneByone(row_i, excel_path, "Sheet1", 
          model_name, training_time, max(val_acc))

        row_i = row_i+1

# write_excel.save_train_results_10_times(excel_path, "Sheet1", 
#           model_name_list, training_time_list, 
#           best_val_acc_list)

### (2) Experiment 2
data augmentation

In [ ]:
repeat_times = 10
lr_rate = 0.0001
model_arc_list = ["DenseNet"]
# model_arc_list = ["SqueezeNet", "DenseNet"]
fine_tune_strategy = "TL_all"
train_dataset_list = [
    # r"F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.1-DA HV"
    # r"F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.2-DA BR"
    # r"F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.3-DA DARK"
    # r"F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.4-DA NI"
    r"F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.5-DA MIX"
]
path_val = r"F:\Tianlong\Data\Deep_Plastic\Data\Exp1\Validation 1"

best_val_acc_list = []
training_time_list = []
model_name_list = []

# path to save weigts
path = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp2\10_times"
# path to save fig
fig_path = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp2\10_times\fig"
# path to save results in a excel file
excel_path=r'F:\Tianlong\Data\Deep_Plastic\Results\Exp2\10_times\models_exp2.xlsx'

In [ ]:
repeat_times

In [ ]:
# train models

for train_dataset in train_dataset_list:
  path_train = train_dataset
  # load datasets
  ds_train, ds_val = tvt_GC.import_dataset_train_val(
      path_train, path_val, image_height, image_width, batch_size)
  # autotune
  AUTOTUNE = tf.data.AUTOTUNE
  ds_train = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  ds_val = ds_val.cache().prefetch(buffer_size=AUTOTUNE)
  
  for model_arc_name in model_arc_list:
    for i in range(repeat_times):
      if train_dataset =="F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.1-DA HV":
        DA_class = "DA_HV"
      if train_dataset =="F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.2-DA BR":
        DA_class = "DA_BR"
      if train_dataset =="F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.3-DA DARK":
        DA_class = "DA_DARK"
      if train_dataset =="F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.4-DA NI":
        DA_class = "DA_NI"
      if train_dataset =="F:\Tianlong\Data\Deep_Plastic\Data\Exp2\Train 2.5-DA MIX":
        DA_class = "DA_MIX"
      
      model_name = model_arc_name + "_" + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + DA_class + "_" + str(i+1) + ".hdf5"
      fig_name_acc = model_arc_name + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + DA_class + "_" + str(i+1) + "_accuracy.jpg"
      fig_name_loss = model_arc_name + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + DA_class + "_" + str(i+1) + "_loss.jpg"
      model_name_list.append(model_name)
      save_model_path = os.path.join(path,model_name)
      save_fig_acc_path = os.path.join(fig_path, fig_name_acc)
      save_fig_loss_path = os.path.join(fig_path, fig_name_loss)

      starttime = datetime.datetime.now()

      if model_arc_name == "SqueezeNet":
        model_result = TL.SqueezeN(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
            save_ckp_path=None, image_height=image_height, image_width=image_width,
            ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)
      if model_arc_name == "DenseNet":
        model_result = TL.DenseN121(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
            save_ckp_path=None, image_height=image_height, image_width=image_width,
            ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)
      
      endtime = datetime.datetime.now()
      training_time=round((endtime - starttime).seconds/60)
      print("training_time: ", training_time)
      training_time_list.append(training_time)
      val_acc = model_result.history['val_accuracy']
      print("Best Validation Accuracy is", max(val_acc))
      best_val_acc_list.append(max(val_acc))
      plot.save_plot_hist(model_result, save_fig_acc_path, save_fig_loss_path)

write_excel.save_train_results_10_times(excel_path, "Sheet1", 
          model_name_list, training_time_list, 
          best_val_acc_list)

### (3) Experiment 3

1. adding images

2. adding images_DA_MIX

3. all images_DA_MIX

In [ ]:
lr_rate = 0.0001
model_arc_list = ["DenseNet"]
# model_arc_list = ["SqueezeNet", "DenseNet"]
fine_tune_strategy = "TL_all"
train_dataset_list = [
    # r"F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Train 3.1-adding_images",
    # r"F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Train 3.2_adding_images_with_DA_MIX",
    # r"F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Train 3.3_all_images_with_DA_MIX",
    r"F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Train 3.4-ANI_DA_HV_ALL"
]
path_val = r"F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Validation 2"

best_val_acc_list = []
training_time_list = []
model_name_list = []

# path to save weigts
path = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp3\10_times"
# path to save fig
fig_path = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp3\10_times\fig"
# path to save results in a excel file
excel_path=r'F:\Tianlong\Data\Deep_Plastic\Results\Exp3\10_times\models_exp3.xlsx'

In [ ]:
# train models
repeat_times = 9

row_i=1


for train_dataset in train_dataset_list:
  path_train = train_dataset
  # load datasets
  ds_train, ds_val = tvt_GC.import_dataset_train_val(
      path_train, path_val, image_height, image_width, batch_size)
  # autotune
  AUTOTUNE = tf.data.AUTOTUNE
  ds_train = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  ds_val = ds_val.cache().prefetch(buffer_size=AUTOTUNE)
  
  for model_arc_name in model_arc_list:
    for i in range(repeat_times):
      if train_dataset == "F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Train 3.1-adding_images":
        train_class = "train3.1"
      if train_dataset == "F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Train 3.2_adding_images_with_DA_MIX":
        train_class = "train3.2"
      if train_dataset =="F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Train 3.3_all_images_with_DA_MIX":
        train_class = "train3.3"
      if train_dataset =="F:\Tianlong\Data\Deep_Plastic\Data\Exp3\Train 3.4-ANI_DA_HV_ALL":
        train_class = "train3.4"

      
      model_name = model_arc_name + "_" + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + train_class + "_" + str(i+1) + ".hdf5"
      fig_name_acc = model_arc_name + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + train_class + "_" + str(i+1) + "_accuracy.jpg"
      fig_name_loss = model_arc_name + fine_tune_strategy +"_lr_" + str(lr_rate) + "_" + train_class + "_" + str(i+1) + "_loss.jpg"
      model_name_list.append(model_name)
      save_model_path = os.path.join(path,model_name)
      save_fig_acc_path = os.path.join(fig_path, fig_name_acc)
      save_fig_loss_path = os.path.join(fig_path, fig_name_loss)

      starttime = datetime.datetime.now()

      if model_arc_name == "SqueezeNet":
        model_result = TL.SqueezeN(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
            save_ckp_path=None, image_height=image_height, image_width=image_width,
            ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)
      if model_arc_name == "DenseNet":
        model_result = TL.DenseN121(num_classes, fine_tune_strategy, class_weights=class_weights_train, save_model_path=save_model_path, 
            save_ckp_path=None, image_height=image_height, image_width=image_width,
            ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)
      
      endtime = datetime.datetime.now()
      training_time=round((endtime - starttime).seconds/60)
      print("training_time: ", training_time)
      # training_time_list.append(training_time)
      
      val_acc = model_result.history['val_accuracy']
      print("Best Validation Accuracy is", max(val_acc))
      # best_val_acc_list.append(max(val_acc))
      
      plot.save_plot_hist(model_result, save_fig_acc_path, save_fig_loss_path)

      write_excel.save_train_results_oneByone(row_i, excel_path, "Sheet1", 
          model_name, training_time, max(val_acc))
      
      row_i = row_i+1

# write_excel.save_train_results_10_times(excel_path, "Sheet1", 
#           model_name_list, training_time_list, 
#           best_val_acc_list)

### Tensorboard

In [ ]:
# load the tensorboard

%load_ext tensorboard

# if the tensorboard page on VS Code is not so clear, 
# you can type this (localhost:6006) on web browser after executing this code 